In [1]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

In [5]:
# Load the data
df = pd.read_csv(r'C:\Users\24707\Downloads\202206-divvy-tripdata\202206-divvy-tripdata.csv')

# Remove rows with 'casual' in the 'member_casual' column
df = df[df['member_casual'] != 'casual']

# Drop rows with NaN values
df = df.dropna()

# Create a graph using NetworkX
G = nx.from_pandas_edgelist(df, source='start_station_id', target='end_station_id')

# Remove nodes with NaN values
G.remove_nodes_from(list(n for n in nx.isolates(G)))

# Create edge index for PyTorch Geometric
#edge_index = torch.tensor([list(map(int, edge)) for edge in G.edges], dtype=torch.long).t().contiguous()

# Create a mapping of node names to integer indices
node_mapping = {node: i for i, node in enumerate(G.nodes)}

# Now create edge index with this mapping
edge_index = torch.tensor([[node_mapping[edge[0]], node_mapping[edge[1]]] for edge in G.edges], dtype=torch.long).t().contiguous()


# Suppose you have feature data for each station
# This can be the historical average demand for each station or other features
# Replace this with your real features
x = torch.tensor(np.random.normal(size=(len(G.nodes), 10)), dtype=torch.float)

# If you have target labels for nodes, for example, the future demand
# Again replace this with your real labels
y = torch.tensor(np.random.normal(size=(len(G.nodes),)), dtype=torch.float)

data = Data(x=x, edge_index=edge_index, y=y)

class STGCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(STGCN, self).__init__()
        torch.manual_seed(10000)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = F.relu(self.fc(x))

        return x

model = STGCN(num_node_features=10, hidden_channels=16, num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Define the MSELoss function
criterion = torch.nn.MSELoss()

# # Create a list to store the losses
# losses = []

for epoch in range(400):
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data)  # Perform a single forward pass.
    loss = criterion(out.squeeze(), data.y)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    
#     # Convert to numpy arrays for printing
#     loss_np = loss.detach().numpy()
    
#     # Store the loss value for this epoch
#     losses.append(loss_np)
    
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')
    
# # After training, plot the loss per epoch
# plt.plot(losses)
# plt.xlabel('Epoch')
# plt.ylabel('MSE Loss')
# plt.title('MSE Loss per Epoch in STGCN Training')
# plt.show()


Epoch: 1, Loss: 1.0658048391342163
Epoch: 2, Loss: 1.0330719947814941
Epoch: 3, Loss: 0.9977315664291382
Epoch: 4, Loss: 0.9963330030441284
Epoch: 5, Loss: 0.9915648698806763
Epoch: 6, Loss: 0.9884036183357239
Epoch: 7, Loss: 0.9847750663757324
Epoch: 8, Loss: 0.9828371405601501
Epoch: 9, Loss: 0.9834375977516174
Epoch: 10, Loss: 0.9827473163604736
Epoch: 11, Loss: 0.9829234480857849
Epoch: 12, Loss: 0.9830668568611145
Epoch: 13, Loss: 0.9834911227226257
Epoch: 14, Loss: 0.9828062057495117
Epoch: 15, Loss: 0.983121395111084
Epoch: 16, Loss: 0.9830289483070374
Epoch: 17, Loss: 0.9828581213951111
Epoch: 18, Loss: 0.9828620553016663
Epoch: 19, Loss: 0.9828581213951111
Epoch: 20, Loss: 0.9827049970626831
Epoch: 21, Loss: 0.9828581213951111
Epoch: 22, Loss: 0.9828581213951111
Epoch: 23, Loss: 0.9828581213951111
Epoch: 24, Loss: 0.9828581213951111
Epoch: 25, Loss: 0.9828569889068604
Epoch: 26, Loss: 0.9828581213951111
Epoch: 27, Loss: 0.9828581213951111
Epoch: 28, Loss: 0.982925295829773
Epo

Epoch: 226, Loss: 0.9828581213951111
Epoch: 227, Loss: 0.9828581213951111
Epoch: 228, Loss: 0.9828581213951111
Epoch: 229, Loss: 0.9828581213951111
Epoch: 230, Loss: 0.9828581213951111
Epoch: 231, Loss: 0.9828581213951111
Epoch: 232, Loss: 0.9828581213951111
Epoch: 233, Loss: 0.9828581213951111
Epoch: 234, Loss: 0.9828581213951111
Epoch: 235, Loss: 0.9828581213951111
Epoch: 236, Loss: 0.9828581213951111
Epoch: 237, Loss: 0.9828693866729736
Epoch: 238, Loss: 0.9828581213951111
Epoch: 239, Loss: 0.9828581213951111
Epoch: 240, Loss: 0.9828581213951111
Epoch: 241, Loss: 0.9828581213951111
Epoch: 242, Loss: 0.9828581213951111
Epoch: 243, Loss: 0.9828581213951111
Epoch: 244, Loss: 0.9828581213951111
Epoch: 245, Loss: 0.9828581213951111
Epoch: 246, Loss: 0.9828581213951111
Epoch: 247, Loss: 0.9828581213951111
Epoch: 248, Loss: 0.9828581213951111
Epoch: 249, Loss: 0.9828581213951111
Epoch: 250, Loss: 0.9828581213951111
Epoch: 251, Loss: 0.9828581213951111
Epoch: 252, Loss: 0.9828581213951111
E